In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
books=pd.read_csv('/kaggle/input/7k-books-with-metadata/books.csv')

In [ ]:
books.head(2)

In [ ]:
books=books.drop(columns=['isbn10','subtitle','published_year','num_pages','ratings_count','thumbnail'])

In [ ]:
books.shape

In [ ]:
books.isnull().sum()

In [ ]:
books.dropna(inplace=True)

In [ ]:
books.duplicated().sum()

In [ ]:
books.shape

In [ ]:
books=books.reset_index(drop=True)
books.head(2)

In [ ]:
books['categories'] = books['categories'].apply(lambda x:x.split())
books['description'] = books['description'].apply(lambda x:x.split())
books['authors'] = books['authors'].apply(lambda x:x.replace(" ",'').split())

In [ ]:
books.head(2)

In [ ]:
books['tags']=books['description']+books['authors']+books['categories']

In [ ]:
books['tags'][0]

In [ ]:
books['tags']=books['tags'].apply(lambda x: " ".join(x))
books['tags'][0]

In [ ]:
df=books.drop(columns=['authors','categories','description'],axis=1)
df

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
df['tags']=df['tags'].apply(stem)

In [ ]:
x=df['tags'].values.tolist()
y=''
for i in x:
    y=y+i
len(y)

In [ ]:
len(set(y.split()))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=50000,stop_words='english')

In [ ]:
vector = cv.fit_transform(df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
def recommend(book):
        index = df[df['title']== book].index[0]
        distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x: x[1])
        for i in distances[1:6]:
            b=df.iloc[i[0]].title
            print(b)

In [ ]:
recommend('Data Structures and Algorithms in C++')

In [ ]:
df[df['title']== 'Insights']

In [ ]:
df[df['average_rating']>=4.74].shape

In [ ]:
    for i in range(8):
        index=books[books['average_rating']==5].index[i]
        print(books[books['average_rating']==5].index[i])
        print(books.iloc[index].isbn13)
        print(books.iloc[index].title)

In [ ]:
import pickle

In [ ]:
pickle.dump(df,open('book_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
pickle.dump(df.to_dict(),open('book_dict.pkl','wb'))